# ConnectorX SQLite Compatibility

Remove notnull constraint, replace 'nan' with NULL's, as experiment towards compatibility.

In [29]:
import connectorx as cx
import pandas as pd
from sqlalchemy import create_engine

In [30]:
!cp testing_err_SQ00014613.sqlite testing_err_fixed_SQ00014613.sqlite

In [31]:
db_filename = "testing_err_fixed_SQ00014613.sqlite"
sqlite_err_conn = create_engine(f"sqlite:///{db_filename}").connect()
connectorx_err_conn = f"sqlite://{db_filename}"

In [32]:
# known error
query = """
select * from cytoplasm
"""
cx.read_sql(conn=connectorx_err_conn, query=query, return_type="pandas")

RuntimeError: Invalid column type Text at index: 61, name: Cytoplasm_Correlation_Costes_AGP_DNA

In [34]:
sql_stmt = """
SELECT name, sql FROM sqlite_master WHERE type ='table'
"""
df = pd.read_sql(sql=sql_stmt, con=sqlite_err_conn).to_dict(orient="records")
for table in df:
    table_name = table["name"]
    table_sql = table["sql"]
    print(f"creating new table for {table_name}")
    # alter to rename the original table with prefix
    sqlite_err_conn.execute(
        statement=f"alter table {table_name} rename to orig_{table_name}"
    )
    # create new table with the original's name and removing all not null constraints
    sqlite_err_conn.execute(statement=table_sql.replace("NOT NULL", ""))
    # copy data from original to new table
    sqlite_err_conn.execute(
        statement=f"insert into {table_name} select * from orig_{table_name}"
    )
    # delete original table
    sqlite_err_conn.execute(statement=f"drop table orig_{table_name}")

    # gather cols from table
    col_names = pd.read_sql(
        sql=f"select name from pragma_table_info('{table_name}')", con=sqlite_err_conn
    )["name"].values.tolist()

    for col in col_names:
        # update 'nan' values to NULL
        sqlite_err_conn.execute(
            statement=f"update {table_name} set {col}=NULL where {col}='nan'"
        )

creating new table for Image
creating new table for Nuclei
creating new table for Cytoplasm
creating new table for Cells


In [35]:
pd.read_sql(sql="select * from cytoplasm", con=sqlite_err_conn)[
    "Cytoplasm_Correlation_Costes_AGP_DNA"
]

0            NaN
1       0.998284
2       0.999907
3       0.999916
4       1.000000
          ...   
5867    1.000000
5868    1.000000
5869    0.999926
5870    1.000000
5871    0.998590
Name: Cytoplasm_Correlation_Costes_AGP_DNA, Length: 5872, dtype: float64

In [36]:
# known error
query = """
select * from cytoplasm
"""
cx.read_sql(conn=connectorx_err_conn, query=query, return_type="pandas")

,TableNumber,ImageNumber,ObjectNumber,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_Center_X,Cytoplasm_AreaShape_Center_Y,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_Eccentricity,Cytoplasm_AreaShape_EulerNumber,Cytoplasm_AreaShape_Extent,...,Cytoplasm_Texture_Variance_DNA_5_0,Cytoplasm_Texture_Variance_ER_10_0,Cytoplasm_Texture_Variance_ER_20_0,Cytoplasm_Texture_Variance_ER_5_0,Cytoplasm_Texture_Variance_Mito_10_0,Cytoplasm_Texture_Variance_Mito_20_0,Cytoplasm_Texture_Variance_Mito_5_0,Cytoplasm_Texture_Variance_RNA_10_0,Cytoplasm_Texture_Variance_RNA_20_0,Cytoplasm_Texture_Variance_RNA_5_0
0,fde835b646e9bb00e6590d00cdec5c88,116,1,10338,391.0,0.0,2.528885,0.879587,0.0,0.348292,...,0.129364,2.137279,2.468285,2.070850,1.987288,2.296482,1.956370,3.299157,3.482296,3.302760
1,fde835b646e9bb00e6590d00cdec5c88,116,2,12505,186.0,59.0,2.560177,0.904184,0.0,0.344026,...,0.209131,3.128372,3.042677,3.340692,2.849144,2.787168,3.055035,3.704050,3.473707,3.955501
2,fde835b646e9bb00e6590d00cdec5c88,116,3,6022,2063.0,107.0,2.096936,0.687956,0.0,0.473987,...,0.978044,2.576061,2.555135,2.484019,2.240818,2.235643,2.447526,3.559578,3.295540,3.771766
3,fde835b646e9bb00e6590d00cdec5c88,116,4,7765,179.0,145.0,1.989944,0.824150,0.0,0.369516,...,0.892772,0.768657,0.394493,1.082676,0.889705,0.325929,1.357238,1.367410,0.835436,1.784678
4,fde835b646e9bb00e6590d00cdec5c88,116,5,5604,1169.0,197.0,3.271878,0.906848,0.0,0.302461,...,1.118692,3.709146,2.829589,3.364716,3.358684,3.286858,2.959485,5.326983,4.181589,4.957180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5867,fe8171164089d06ebeda3a454e93a38c,3287,228,5746,424.0,2027.0,2.707400,0.915282,0.0,0.298323,...,1.780132,4.217649,3.507293,4.086303,4.295762,3.507914,4.191847,3.970293,2.837317,4.348051
5868,fe8171164089d06ebeda3a454e93a38c,3287,229,11792,1630.0,2100.0,1.573115,0.660023,0.0,0.482764,...,0.960271,3.221004,3.371353,3.166467,3.126756,3.213101,3.094729,3.845710,3.643518,4.082686
5869,fe8171164089d06ebeda3a454e93a38c,3287,230,13093,145.0,2159.0,2.568904,0.878110,0.0,0.339109,...,0.552542,2.670752,3.010857,2.539699,2.140841,2.324754,2.069712,2.249473,2.502827,2.205429
5870,fe8171164089d06ebeda3a454e93a38c,3287,231,5688,980.0,2159.0,2.219555,0.741893,0.0,0.387202,...,1.727293,1.726927,2.055037,1.774497,1.845742,1.766132,2.132555,2.977602,2.510163,3.336011
